# Personal Expense Tracker

### Add Expenses

In [2]:
from datetime import datetime
import re
import csv

def add_expenses():

    CATEGORIES = ['Travel','Shopping','Groceries','Food & Drink','Personal','Gas','Bills & Utilities','Education']

    # Found and researched a common currency regex for USD
    # Huge regex to filter for the amount pattern for user input, I'll list the order of how the regex pattern is working below.
    # 1. optional spaces - \s*
    # 2. an optional dollar sign assuming USD - \$?
    # 3. optional spaces - \s*
    # 4. required any number of digits OR 1 to 3 digits that are optionally followed by a comma with 3 digits - (?:\d+|\d{1,3}(?:,\d{3})*)
    # 5. an optional period with 1 to 2 digits - (?:\.\d{1,2})?
    # 6. optional spaces - \s*
    AMOUNT_PATTERN = re.compile(r'^\s*\$?\s*(?:\d+|\d{1,3}(?:,\d{3})*)(?:\.\d{1,2})?\s*$')

    addedExpenseAmount = input('Please enter the amount of the expense (i.e. 65,432.43 or $65,432.43): ')
    #If block to determine if the amount input is correctly written and then cleans it up to be parsed
    if AMOUNT_PATTERN.match(addedExpenseAmount):

        try:
            addedExpenseAmount.strip()  #Removes trailing/leading spaces if any
            addedExpenseAmount = float(addedExpenseAmount.replace('$','').replace(',',''))   #Clean the string from $ sign and commas then parse into a float

            #Check if user input negative or zero and avoid adding the expense
            if addedExpenseAmount <= 0:
                print('You entered a negative amount or zero. Can not add to expenses.')
                return None
            
        except ValueError as e:
            print(f'There was a value error in the amount. {e}')
            return None
        except Exception as e:
            print(f'Unexpected Error: {e}')
            return None
    else:
        print('You did not enter a valid expense amount.')
        return None

    dateExpenseSpent = input('You can enter the date of this expense (i.e. 2025-03-20): ')
    #If user cancels input for date default to now
    if(dateExpenseSpent != ''):
        # Try except to check for the correct date format YYYY-MM-DD
        try:
            dateExpenseSpent = datetime.strptime(dateExpenseSpent, '%Y-%m-%d').date()
        except ValueError as e:
            print(f'You did not enter the correct date format. (i.e. 2025-03-20) {e}')
            return None
        except Exception as e:
            print(f'Unexpected Error: {e}')
            return None
    else:
        dateExpenseSpent=datetime.now().date()

    expenseCategory = input(f'You can enter an expense category, options are {', '.join(CATEGORIES)}:')
    #If block to check categories if user cancels defaults to Personal
    if(expenseCategory != '' and not any(iCategory == expenseCategory for iCategory in CATEGORIES)):
        print('You did not enter a valid category.')
        return None
    else:
        expenseCategory='Personal'

    expenseDescription = input('You can enter a description of the expense: ')

    #After all input checks we find/create the file expenses.csv and write in our dict object
    addedExpense = {'amount': addedExpenseAmount, 'date':dateExpenseSpent, 'category': expenseCategory, 'description': expenseDescription}
    print('Expense added successfully!')

    return addedExpense


### Viewing Expenses

In [3]:
def viewExpenses(addedExpenses):

    for expense in addedExpenses:
        print(expense)



### Set and Track Budget

In [4]:
from datetime import datetime
import csv
from pathlib import Path

'''
    budget = float, amount set for the month or monthly budget
    yearMonth = optional date, specific month budget the user can set for 1 month of a year
    monthly = boolean flag, monthly constant budget going forward unless user sets a new one

    We set monthly budgets here with the option of a specific month to be a different budget amount. We keep track of this monthly with a flag to 
    make sure any month's directly influenced by the user, doesn't change all budget's.
'''
def setMonthlyBudget():

    # Found and researched a common currency regex for USD
    # Huge regex to filter for the amount pattern for user input, I'll list the order of how the regex pattern is working below.
    # 1. optional spaces - \s*
    # 2. an optional dollar sign assuming USD - \$?
    # 3. optional spaces - \s*
    # 4. required any number of digits OR 1 to 3 digits that are optionally followed by a comma with 3 digits - (?:\d+|\d{1,3}(?:,\d{3})*)
    # 5. an optional period with 1 to 2 digits - (?:\.\d{1,2})?
    # 6. optional spaces - \s*
    AMOUNT_PATTERN = re.compile(r'^\s*\$?\s*(?:\d+|\d{1,3}(?:,\d{3})*)(?:\.\d{1,2})?\s*$')

    budget = input('How much do you want to set for your monthly budget? (i.e. 65,432.43 or $65,432.43):')
    #If block to determine if the amount input is correctly written and then cleans it up to be parsed
    if AMOUNT_PATTERN.match(budget):

        try:
            budget.strip()  #Removes trailing/leading spaces if any
            budget = float(budget.replace('$','').replace(',',''))   #Clean the string from $ sign and commas then parse into a float

            #Check if user put a negative or zero
            if budget <= 0:
                print('You entered a negative amount or zero. Can not update budget.')
                return None
        except ValueError as e:
            print(f'There was a value error in the amount. {e}')
            return None
        except Exception as e:
            print(f'Unexpected Error: {e}')
            return None
    else:
        print('You did not enter a valid expense amount.')
        return None


    yearMonth = input('If you want this budget for a specific month of a year. Please type a year and month (i.e. 2025-03). Otherwise enter or use ESC to cancel.')
    updatedBudget = {}
    #Check user inputs to match a month and year for setting optional one time budgets
    #We are avoiding changing older budgets from the current month
    if yearMonth:     #User canceled or didn't input if empty.
        try:
            now = datetime.now()
            if(datetime.strptime(yearMonth, '%Y-%m') < datetime(now.year, now.month, 1)):
                print('You can not change a past budget.')
                return None
            else:
                updatedBudget = {'date': datetime.strptime(yearMonth, '%Y-%m').strftime('%Y-%m'), 'budget': budget, 'monthly': False}
        except ValueError as e:
            print(f'You did not enter the correct date format. (i.e. 2025-03) {e}')
            return None
        except Exception as e:
            print(f'Unexpected Error: {e}')
            return None



    #Read/Create the budgets file and put it in a list to manipulate 
    try:
        with open('budgets.csv', 'r', newline='') as budgetsFile:    
            budgetsReader = csv.DictReader(budgetsFile)
            budgetRows = list(budgetsReader)
    except FileNotFoundError as e:
        print(f'File could not be found! creating the file budgets.csv now. {e}')
        budgetsFile = open('budgets.csv', 'w')    #Create the file with write mode
    except Exception as e:
        print(f'Unexpected error occurred: {e}')
    finally:
        budgetsFile.close()



    

    #If the user wanted to set a specific month's budget we read and update the entire file
    if yearMonth:

        #Check if the row for that year month exists, update that row's budget, if not we make one
        if any(datetime.strptime(iRow['date'], '%Y-%m') == updatedBudget['date'] for iRow in budgetRows):
            for row in budgetRows:
                if(updatedBudget['date'] == datetime.strptime(row['date'], '%Y-%m')):
                    row['budget'] = updatedBudget['budget']
                    row['monthly'] = False
        else:
            budgetRows.append(updatedBudget)

        try:
            with open('budgets.csv', 'w', newline='') as budgetsFile:
                budgetWriter = csv.DictWriter(budgetsFile,fieldnames=budgetRows[0].keys())
                budgetWriter.writeheader()      #Write always recreates the file so we write header
                budgetWriter.writerows(budgetRows)
                print('Budget file updated successfully!')
        except Exception as e:
            print(f'Unexpected error occurred: {e}')
            return None
        finally:
            budgetsFile.close()

    #User just wants to update monthly budget's going forward, so we append to the file what the new budget is this month
    else:
        #Check if the file exists before writing to it, to avoid inserting the column headers on every save
        budgetFilePath = Path('budgets.csv')

        #default date and set monthly if the user wanted to set monthly budget
        addedBudget = {'date': datetime.now().strftime('%Y-%m'), 'budget': round(budget, 2), 'monthly': True}

        try:
            with open('budgets.csv', 'a', newline='') as budgetsFile:
                budgetWriter = csv.DictWriter(budgetsFile,fieldnames=addedBudget.keys())
                if not budgetFilePath.exists() or budgetFilePath.stat().st_size == 0:
                    budgetWriter.writeheader()      #Write column names if the file doesn't exist, otherwise just append
                budgetWriter.writerow(addedBudget)
                print('Budget file updated successfully!')
        except Exception as e:
            print(f'Unexpected error occurred: {e}')
            return None
        finally:
            budgetsFile.close()

In [5]:
from datetime import datetime
import csv

def trackMonthlyBudgetExpenses():

    budgetRows = []
    expensesRows = []
    
    #Read the budget file
    try:
        with open('budgets.csv', 'r', newline='') as budgetsFile:
            budgetsReader = csv.DictReader(budgetsFile)
            budgetRows = list(budgetsReader)
    except FileNotFoundError as e:
        print(f'File could not be found! creating the file budgets.csv now. {e}')
        budgetsFile = open('budgets.csv', 'w')   #Create the file with write mode
    except Exception as e:
        print(f'Unexpected error occurred: {e}')


    #Read the expenses file to compare
    try:
        with open('expenses.csv', 'r', newline='') as expensesFile:
            expensesReader = csv.DictReader(expensesFile)
            expensesRows = list(expensesReader)
    except FileNotFoundError as e:
        print(f'File could not be found! creating the file expenses.csv now. {e}')
        expensesFile = open('expenses.csv', 'w')   #Create the file with write mode
    except Exception as e:
        print(f'Unexpected error occurred: {e}')
    finally:
        expensesFile.close()

# or (row['date'] == datetime.now().strftime('%Y-%m'))
    #Looping through the budget file and getting the last monthly budget set by the user, since this is the last time they wanted to change it
    try:
        lastMonthlyBudget = next((float(row['budget']) for row in reversed(budgetRows) if row['monthly'] == 'True' or row['date'] == datetime.now().strftime('%Y-%m')), 0)
    except ValueError as e:
        print(f'Invalid budget amount found in the budget file. Please contact admin for help. {e}')
    except Exception as e:
        print(f'Unexpected error found. {e}')
    print(f'Last set monthly Budget: {lastMonthlyBudget}')


    currMonthlyExpenses = []
    #Looping through the expenses file and filtering based on the current month to get total sum of expenses
    try:
        currMonthlyExpenses = [float(row['amount']) for row in expensesRows if datetime.strptime(row['date'], '%Y-%m-%d').strftime('%Y-%m') == datetime.now().strftime('%Y-%m')]
    except ValueError as e:
        print(f'There is a faulty expense in the expenses file. Skipping this entry. Please ask an admin for help. {e}')
        pass
    except Exception as e:
        print(f'An unexpected error occurred: {e}')
        return None
    currMonthlyExpense = sum(currMonthlyExpenses)
    print(f'Total month\'s expenses so far: {currMonthlyExpense}')


    #Compare current expenses to the last set monthly budget
    if(lastMonthlyBudget < currMonthlyExpense):
        print('You have exceeded your budget!')
    else:
        returnBudget = round(lastMonthlyBudget - currMonthlyExpense, 2)
        print(f'You have {returnBudget} left for the month.')



### Save and Load to expenses file

In [6]:
import csv
from pathlib import Path

def saveCSVExpenses(addedExpenses):

    if len(addedExpenses) > 0:
        #Check if the file exists before writing to it, to avoid inserting the column headers on every save
        expensesFilePath = Path('expenses.csv')

        #Save expenses list added by user in memory from the addExpenses function
        try:
            with open('expenses.csv', 'a', newline='') as expensesFile:    #Create the file with append mode and to avoid overwrite if it exists
                expensesWriter = csv.DictWriter(expensesFile, fieldnames = addedExpenses[0].keys())
                if not expensesFilePath.exists() or expensesFilePath.stat().st_size == 0:
                    expensesWriter.writeheader()    #Write column names if the file doesn't exist, otherwise just append
                expensesWriter.writerows(addedExpenses)      #Write all rows values
                print('Expenses saved successfully!')
        except Exception as e:
            print(f'Unexpected Error occurred: {e}')
            return None
        finally:
            expensesFile.close()
    else:
        print('No expenses added. Nothing to save.')

In [25]:
import csv

def loadCSVExpenses():

    #Check for the expenses file and load all expenses back to user 
    try:
        with open('expenses.csv', 'r', newline='') as expensesFile:
            expensesReader = csv.DictReader(expensesFile)
            expensesRows = list(expensesReader)
            print('Expenses loaded successfully!')
            for expense in expensesRows:
                print(expense)
    except FileNotFoundError as e:
        print(f'File could not be found! creating the file expenses.csv now. {e}')
        expensesFile = open('expenses.csv', 'w')   #Create the file with write mode
    finally:
        expensesFile.close()

    return expensesRows

### Interactive Menu for User Input

In [23]:
def interactiveMenuInput():

    INPUT_OPTIONS = ['Add expense', 'View expenses', 'Track budget', 'Save expenses', 'Exit']
    addedExpenses = []
    loadCSVExpenses()

    #We want to create a loop of the options until the user exits and is done with adding, saving, loading, etc.
    while True:
    
        print('Hi. What would you like to do?')
        for i, option in enumerate(INPUT_OPTIONS, start=1):
            print(f'{i}. {option}')

        
        choice = input('Enter the number of your choice: ')
        if(choice == ''):    #User canceled input save just in case
            print('User canceled.')
            saveCSVExpenses(addedExpenses)
            break
        try:
            choice = int(choice)    #Check if User input an integer, if not skip and default to 0, start over
        except TypeError:
            print('You did not enter a valid number or an integer. Please try again.')
            choice = 0
        except ValueError:
            print('You did not enter a number. Please try again.')
            choice = 0
        except Exception as e:
            print(f'Unexpected error occurred: {e}')
            return None


        choiceName = ''
        if 1 <= choice <= len(INPUT_OPTIONS):
            choiceName = INPUT_OPTIONS[choice - 1]
            print(f'You chose: {choiceName} \n')
        else:
            print('Invalid choice. Please enter a valid number.')
        
        

        #if the choice is a valid name, then we can continue with this series of IF statements, otherwise only the above code runs
        if(choiceName == 'Add expense'):
            
            addedExpenses.append(add_expenses())

        elif(choiceName == 'View expenses'):

            #We are allowing user to select between loading all expenses or viewing the current added expenses in memory
            VIEW_OPTIONS = ['Load expenses', 'View expenses']
            for i, option in enumerate(VIEW_OPTIONS, start=1):
                print(f'{i}. {option}')

            viewChoice = input('Would you like to view current added expenses or load all expenses from a file? ')
            if(viewChoice == ''):    #User canceled input
                break
            try:
                viewChoice = int(viewChoice)
            except TypeError as e:
                print(f'You did not enter a valid number or an integer. Please try again. {e}')
                viewChoice = 0

            if 1 <= viewChoice <= len(VIEW_OPTIONS):
                viewName = VIEW_OPTIONS[viewChoice - 1]
                print(f'You selected: {viewName} \n')
            else:
                print('Invalid choice. Please enter a valid number.')


            #Check which option the user picked above and run that method
            if(viewName == 'Load expenses'):
                loadCSVExpenses()

            elif(viewName == 'View expenses'):
                viewExpenses(addedExpenses)

        elif(choiceName == 'Track budget'):

            #We are allowing user to select between setting budgets or tracking the current month's budget
            BUDGET_OPTIONS = ['Set budget', 'Track budget']
            for i, option in enumerate(BUDGET_OPTIONS, start=1):
                print(f'{i}. {option}')

            budgetChoice = input('Would you like to set your monthly budget or track your budget and expenses for the month from a file?')
            if(budgetChoice == ''):    #User canceled input
                break

            try:
                budgetChoice = int(budgetChoice)
            except TypeError as e:
                print(f'You did not enter a valid number or an integer. Please try again. {e}')
                budgetChoice = 0

            if 1 <= budgetChoice <= len(BUDGET_OPTIONS):
                budgetName = BUDGET_OPTIONS[budgetChoice - 1]
                print(f'You selected: {budgetName} \n')
            else:
                print('Invalid choice. Please enter a valid number.')


            #Check which option the user picked above and run that method
            if(budgetName == 'Set budget'):
                setMonthlyBudget()

            elif(budgetName == 'Track budget'):
                trackMonthlyBudgetExpenses()


        elif(choiceName == 'Save expenses'):

            saveCSVExpenses(addedExpenses)

        elif(choiceName == 'Exit'):

            saveCSVExpenses(addedExpenses)
            break



In [26]:
interactiveMenuInput()

Expenses loaded successfully!
{'amount': '20.0', 'date': '2025-10-29', 'category': '', 'description': 'first test adding expenses without a category'}
{'amount': '25.0', 'date': '2025-10-29', 'category': 'Groceries', 'description': 'second test with category chosen'}
{'amount': '30.56', 'date': '2025-10-29', 'category': 'Groceries', 'description': 'testing for dollar sign currency input '}
{'amount': '45567.12', 'date': '2020-03-18', 'category': 'Education', 'description': 'testing past expenses adding fails'}
{'amount': '56765.21', 'date': '2025-10-29', 'category': 'Education', 'description': 'testing past dates not adding'}
{'amount': '34.56', 'date': '2025-10-29', 'category': 'Gas', 'description': 'testing view current added expenses'}
{'amount': '1235.78', 'date': '2025-12-29', 'category': 'Personal', 'description': 'testing incorrect inputs adding to file'}
{'amount': '43573.0', 'date': '2025-10-30', 'category': 'Personal', 'description': 'testing save'}
{'amount': '21354.0', 'dat